In [1]:
import tabula
from tabula import read_pdf

In [2]:
file_path = 'guardian1.pdf'
dict_list = tabula.read_pdf(file_path, pages='all')


Got stderr: Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:53:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 10, 2023 1:5

In [3]:
print(len(dict_list))


1


In [4]:
lst = []
for ind, tab in enumerate(dict_list):
    if any('Patient Name' in col for col in tab.columns):
        new_columns_name = {}
        for i in range(len(tab.columns)):
            old_name = tab.columns[i]
            new_name = f'columns{i + 1}'
            new_columns_name[old_name] = new_name
            tab = tab.rename(columns=new_columns_name)
        lst.append(tab.to_dict('records'))
    if len(lst) == 0:
        if any('Claim Number' in col for col in tab.columns):
            new_columns_name = {}
            for i in range(len(tab.columns)):
                old_name = tab.columns[i]
                new_name = f'columns{i + 1}'
                new_columns_name[old_name] = new_name
                tab = tab.rename(columns=new_columns_name)
            lst.append(tab.to_dict('records'))     
        

In [5]:
temp_lst = []
for i in range(len(lst)):
    for obj in lst[i]:
        if type(obj.get('columns1')) != float:
            temp_lst.append(obj)

filtered_lst = []
for i in range(len(temp_lst)):
    if not temp_lst[i]['columns1'].startswith('Patient Name') and not temp_lst[i]['columns1'].startswith('Planholder') and not temp_lst[i]['columns1'].startswith('Line Submitted') and not temp_lst[i]['columns1'].startswith('No.'):
        filtered_lst.append(temp_lst[i])
print("filtered_lst>>>>>>>>>>>>>>>>>",filtered_lst)
filtered_lst

filtered_lst>>>>>>>>>>>>>>>>> [{'columns1': '1 D9230/Analgesia/N20', 'columns2': 'FM 05/22/23', 'columns3': '30.00 28.00', 'columns4': '0.00', 'columns5': '80% 0.00'}, {'columns1': '2 D2392/Pst Composite2', 'columns2': '03 05/22/23', 'columns3': '152.00 109.00', 'columns4': '109.00', 'columns5': '50.00 80% 47.20'}, {'columns1': '3 D2392/Pst Composite2', 'columns2': '14 05/22/23', 'columns3': '152.00 109.00', 'columns4': '109.00', 'columns5': '80% 87.20'}]


[{'columns1': '1 D9230/Analgesia/N20',
  'columns2': 'FM 05/22/23',
  'columns3': '30.00 28.00',
  'columns4': '0.00',
  'columns5': '80% 0.00'},
 {'columns1': '2 D2392/Pst Composite2',
  'columns2': '03 05/22/23',
  'columns3': '152.00 109.00',
  'columns4': '109.00',
  'columns5': '50.00 80% 47.20'},
 {'columns1': '3 D2392/Pst Composite2',
  'columns2': '14 05/22/23',
  'columns3': '152.00 109.00',
  'columns4': '109.00',
  'columns5': '80% 87.20'}]

In [6]:
new_lst = []
new_dict = {}
for obj in filtered_lst:
    columns3 = obj['columns3'].split()
    print(columns3)
    columns4 = obj['columns4'].split()
    columns5 = obj['columns5'].split()

    if 'columns6' in obj:
        columns6 = str(obj['columns6']).split()
        columns4 = obj['columns4'].split()
        if len(columns6) == 3 and len(columns4) == 3:
            new_dict = {
                'SubmittedADACodesDescription': obj["columns1"],
                'AltCode': obj["columns2"],
                'ToothNo': obj["columns3"],
                'DateOfService': columns4[0],
                'SubmittedCharge': columns4[1],
                'ConsideredCharge': columns4[2],
                'CoveredCharge': obj["columns5"],
                'DeductibleAmount': columns6[0],
                'CoveragePercent': columns6[1],
                'BenefitAmount': columns6[2]
            }
        elif len(columns6) == 2 and len(columns4) == 3:
            new_dict = {
                'SubmittedADACodesDescription': obj["columns1"],
                'AltCode': obj["columns2"],
                'ToothNo': obj["columns3"],
                'DateOfService': columns4[0],
                'SubmittedCharge': columns4[1],
                'ConsideredCharge': columns4[2],
                'CoveredCharge': obj["columns5"],
                'DeductibleAmount': "",
                'CoveragePercent': columns6[0],
                'BenefitAmount': columns6[1]
            }
        elif len(columns6) == 1 and len(columns4) == 4:
            new_dict = {
                'SubmittedADACodesDescription': obj["columns1"],
                'AltCode': obj["columns2"],
                'ToothNo': obj["columns3"],
                'DateOfService': columns4[0],
                'SubmittedCharge': columns4[1],
                'ConsideredCharge': columns4[2],
                'CoveredCharge': columns4[3],
                'DeductibleAmount': "",
                'CoveragePercent': obj["columns5"],
                'BenefitAmount': str(obj["columns6"])
            }
    elif len(columns3) == 3:
        columns2 = obj['columns2'].split()
        columns4 = obj['columns4'].split()
        new_dict = {
            'SubmittedADACodesDescription': obj["columns1"],
            'AltCode': "",
            'ToothNo': columns2[0],
            'DateOfService': columns2[1],
            'SubmittedCharge': columns3[0],
            'ConsideredCharge': columns3[1],
            'CoveredCharge': columns3[2],
            'DeductibleAmount': "",
            'CoveragePercent': columns4[0],
            'BenefitAmount': str(columns4[1])
        }
    elif len(obj) == 5:
        columns5 = obj['columns5'].split()
        columns2 = ""
        if obj['columns2'].startswith('nan'):
            obj['columns2'] = ""
        else:
            columns2 = obj['columns2'].split()

        columns4 = obj['columns4'].split()
        if len(columns3) == 1:
            new_dict = {
                'SubmittedADACodesDescription': obj["columns1"],
                'AltCode': "",
                'ToothNo': obj['columns3'],
                'DateOfService': columns4[0],
                'SubmittedCharge': columns4[1],
                'ConsideredCharge': columns4[2],
                'CoveredCharge': columns4[3],
                'DeductibleAmount': "",
                'CoveragePercent': columns5[0],
                'BenefitAmount': str(columns5[1])
            }
            print("new_dict>>>>>>>>>>>>>>>", new_dict)

        elif len(columns5) == 2:
            new_dict = {
                'SubmittedADACodesDescription': obj["columns1"],
                'AltCode': "",
                'ToothNo': obj['columns2'],
                'DateOfService': columns3[0],
                'SubmittedCharge': columns3[1],
                'ConsideredCharge': columns3[2],
                'CoveredCharge': columns3[3],
                'DeductibleAmount': "",
                'CoveragePercent': columns4[0],
                'BenefitAmount': str(columns4[1])
            }
            print("new_dict>>>>>>>>>>>>>>>", new_dict)

        elif len(columns5) == 3:
            new_dict = {
                'SubmittedADACodesDescription': obj["columns1"],
                'AltCode': "",
                'ToothNo': columns2[0],
                'DateOfService': columns2[1],
                'SubmittedCharge': columns3[0],
                'ConsideredCharge': columns3[1],
                'CoveredCharge': obj["columns4"],
                'DeductibleAmount': columns5[0],
                'CoveragePercent': columns5[1],
                'BenefitAmount': str(columns5[2])
            }
            print("new_dict>>>>>>>>>>>>>>>", new_dict)


    else:
        columns3 = obj['columns3'].split()
        columns4 = obj['columns4'].split()
        new_dict = {
            'SubmittedADACodesDescription': obj["columns1"],
            'AltCode': "",
            'ToothNo': obj["columns2"],
            'DateOfService': columns3[0],
            'SubmittedCharge': columns3[1],
            'ConsideredCharge': columns3[2],
            'CoveredCharge': columns3[3],
            'DeductibleAmount': "",
            'CoveragePercent': columns4[0],
            'BenefitAmount': columns4[1]
        }

    new_lst.append(new_dict)
print("new_lst>>>>>>>>>>>>>", new_lst)


['30.00', '28.00']


IndexError: list index out of range